In [ ]:
# umbridge server
import nest_asyncio
nest_asyncio.apply()
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.special as special
from mpl_toolkits.mplot3d import axes3d
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
from scipy.linalg import block_diag
import umbridge
from scipy import linalg

class PDEModel(umbridge.Model):
    def __init__(self):
        super().__init__("forward")
        self.C= np.diag([2, 0, 0, 0, 2, 2])
        self.T = np.ones((6, 6))
        self.T[0] = 1/np.sqrt(6) * np.ones((1,6))
        self.T[1, 0:3] = - 1/np.sqrt(6) * np.ones((1,3))
        self.T[1, 3:6] = 1/np.sqrt(6) * np.ones((1,3))
        self.T[2] = [1/2, -1/2, 0, -1/2, 1/2, 0]
        self.T[3] = [1/ np.sqrt(12), 1/ np.sqrt(12), -1/ np.sqrt(3), -1/ np.sqrt(12), -1/ np.sqrt(12), 1/ np.sqrt(3) ]
        self.T[4] = [-1/2, 1/2, 0, -1/2, 1/2, 0]
        self.T[5] = [-1/ np.sqrt(12), -1/ np.sqrt(12), 1/ np.sqrt(3), -1/ np.sqrt(12), -1/ np.sqrt(12), 1/ np.sqrt(3) ]
        self.n = 6
        self.external_noise = [0] * 4  # Initialize with the required size (4 elements)
        self.U_AC = 325
        self.U_DC = 700

        
        
        
    def get_input_sizes(self, config):
        return [1, 1, 1, 1, 1, 1, 6, 6, 1, 1] 
 # for the values 6 is for R_arm, L_arm etc., 6 is for u_1:6, initial conditions for x, angular frequency, time  
    
    def get_output_sizes(self, config):
        return [6]  # for the output variables x_1:6
        
    def __call__(self, parameters, config):
        R_arm = parameters [0][0]
        R_DC = parameters [0][1]
        R_AC = parameters [0][2]
        L_arm = parameters [0][3]
        L_DC = parameters [0][4]
        L_AC = parameters [0][5]
        u_parameter = parameters [0][6]
        x_initial = parameters [0][7]
        omega = parameters[0][8]
        T = parameters[0][9]
        matrix_A = self.a(self.n, R_arm, R_DC, L_arm, L_DC, R_AC, L_AC)
        matrix_B = self.b(self.n, L_arm, L_DC)
        matrix_F = self.f(self.n, L_arm, L_DC)
        #z_vector = self.z(T, omega)
        sol = solve_ivp(self.MMC, [0, T], x_initial, args=(matrix_A, matrix_B, matrix_F, u_parameter, config), method='RK45', t_eval=[T])
        result = [sol.y[0], sol.y[1], sol.y[2], sol.y[3], sol.y[4], sol.y[5]]
        return [result]



    # A matrix
    def a(self, n, R_arm, R_DC, L_arm, L_DC, R_AC, L_AC, config):
        A = np.zeros((self.n, self.n))
        np.fill_diagonal(A, [0, -(R_arm+3*R_DC)/(L_arm+3*L_DC), -(R_arm)/(L_arm), -(R_arm)/(L_arm), -(R_arm+2*R_AC)/(L_arm+2*L_AC), -(R_arm+2*R_AC)/(L_arm+2*L_AC)])
        return A  

    # B matrix
    def b(self, n, L_arm, L_DC, config):
        B = np.zeros((self.n, self.n))
        np.fill_diagonal(B, [0, -(1)/(L_arm+3*L_DC), -(1)/(L_arm), -(1)/(L_arm), -(1)/(L_arm+2*L_AC), -(1)/(L_arm+2*L_AC)])
        return B  

    # F matrix
    def f(self, n, L_arm, L_DC, config):
        F = np.zeros((self.n , self.n -2))
        np.fill_diagonal(F, [0, -(R_arm+3*R_DC)/(L_arm+3*L_DC), -(R_arm)/(L_arm), -(R_arm)/(L_arm), -(R_arm+2*R_AC)/(L_arm+2*L_AC), -(R_arm+2*R_AC)/(L_arm+2*L_AC)])
        return F  

    def z(self, T, omega, config):
        
        for k in range(1, 4):
            self.external_noise[k-1]= self.U_AC * np.cos (omega * T - (2 * np.pi * (k-1))/3)

        self.external_noise[3] = self.U_DC
        return self.external_noise
            

    # system of equations
    def MMC(self, T, x, A, B, F, u, config):
        
        dxdt = A @ x + B @ u + F @ self.external_noise

        return dxdt

   
    def supports_evaluate(self):
        return True

pde_model = PDEModel()
umbridge.serve_models([pde_model], 4242) # start model server

In [ ]:
def __call__(self, parameters, config):
        T = parameters[0][0]
        u0_prey = parameters[0][1] 
        u0_predator = parameters[0][2]
        theta1 = parameters[0][3]
        theta2 = parameters[0][4]
        theta12 = parameters[0][5]
        theta21 = parameters[0][6]
        u0_bar = [u0_prey, u0_predator ]
        sol = solve_ivp(self.lotka_volterra, [0, T], u0_bar, args=(theta1, theta2, theta12, theta21, config), method='RK45', t_eval=[T])
        result = [float(sol.y[0]), float(sol.y[1])]
        return [result]

    #solve the Lotka-Volterra model and consider u1 as prey, u2 as predator
    def lotka_volterra(self, t, u, theta1, theta2, theta12, theta21, config):
        u1, u2 = u
        du1dt = theta1 * u1 - theta12 * u1 * u2
        du2dt = theta21 * u1 * u2 - theta2 * u2
        return [du1dt, du2dt]


In [ ]:
def ode_system(t, y):
    return -A @ y + np.cos(omega * t)

# Initial conditions (example)
y0 = np.zeros(6)  # Initial conditions for y (6-dimensional vector)

# Solve the ODE
solution = solve_ivp(ode_system, t_span, y0, t_eval=t_eval)

In [ ]:
import numpy as np

matrix = np.ones((6, 6))
matrix[0] = 1/np.sqrt(6) * np.ones((1,6))
matrix[1, 0:3] = - 1/np.sqrt(6) * np.ones((1,3))
matrix[1, 3:6] = 1/np.sqrt(6) * np.ones((1,3))
matrix[2] = [1/2, -1/2, 0, -1/2, 1/2, 0]
matrix[3] = [1/ np.sqrt(12), 1/ np.sqrt(12), -1/ np.sqrt(3), -1/ np.sqrt(12), -1/ np.sqrt(12), 1/ np.sqrt(3) ]
matrix[4] = [-1/2, 1/2, 0, -1/2, 1/2, 0]
matrix[5] = [-1/ np.sqrt(12), -1/ np.sqrt(12), 1/ np.sqrt(3), -1/ np.sqrt(12), -1/ np.sqrt(12), 1/ np.sqrt(3) ]
matrix

In [ ]:
import numpy as np

# Creating a 3x3 matrix of ones
matrix = np.ones((3, 3))
print("Original matrix:")
print(matrix)

# Changing the second row (index 1) to [5, 5, 5]
matrix[1] = [5, 5, 5]
print("\nMatrix after changing the second row:")
print(matrix)


In [ ]:
import numpy as np

# Define the size of the matrix
n = 6  # Set to 6, or any other size

# Create an n x n matrix of ones
matrix = np.ones((n, n))

# Apply transformations to each row based on the value of n
matrix[0] = 1 / np.sqrt(n) * np.ones((1, n))
matrix[1, 0:n//2] = -1 / np.sqrt(n) * np.ones((1, n//2))
matrix[1, n//2:n] = 1 / np.sqrt(n) * np.ones((1, n//2))
matrix[2] = [1/2, -1/2, 0, -1/2, 1/2, 0] if n == 6 else [1/2, -1/2] + [0] * (n - 4) + [-1/2, 1/2, 0]
matrix[3] = [1/np.sqrt(12), 1/np.sqrt(12), -1/np.sqrt(3), -1/np.sqrt(12), -1/np.sqrt(12), 1/np.sqrt(3)] if n == 6 else [1/np.sqrt(12)] * 2 + [-1/np.sqrt(3)] + [-1/np.sqrt(12)] * 2 + [1/np.sqrt(3)]
matrix[4] = [-1/2, 1/2, 0, -1/2, 1/2, 0] if n == 6 else [-1/2, 1/2] + [0] * (n - 4) + [-1/2, 1/2, 0]
matrix[5] = [-1/np.sqrt(12), -1/np.sqrt(12), 1/np.sqrt(3), -1/np.sqrt(12), -1/np.sqrt(12), 1/np.sqrt(3)] if n == 6 else [-1/np.sqrt(12)] * 2 + [1/np.sqrt(3)] + [-1/np.sqrt(12)] * 2 + [1/np.sqrt(3)]

# Display the matrix
print(matrix)


In [ ]:
def h(n,k):
    b = np.zeros((n - 1)**2)
    b[:(n-1)] = -(g/k) * (h_val**2) - T_0
    b[(n-1):(n - 1)**2]=  -(g/k) * (h_val**2)
return b





In [ ]:
b = np.zeros((5 - 1)**2)
b

In [ ]:
np.zeros((2,2))

In [ ]:
import numpy as np

# Create a 4x4 matrix of ones
matrix = np.ones((4, 4))

# Fill the diagonal with a specific value (e.g., 5)
np.fill_diagonal(matrix, [55,5,5,5,5])
x = np.array(2, 3, 4, 1)
print(matrix)
print(type(matrix))
print(matrix @ x)

In [ ]:
[0] * 4  # Initialize with the required size (3 elements)